Would it be possible for you to send me over a tsv file with the following quantities : Image name, lambda value, rate in bpp, psnr in dB, number of pixels ?

In [ ]:
import pandas as pd
from PIL import Image

from coolchic.utils.paths import DATA_DIR, RESULTS_DIR

csvs = RESULTS_DIR / "exps" / "delta-hn/ups-best-orange"
# csvs = RESULTS_DIR / "exps" / "copied/no-cchic/orange-best"
all_csvs = list(csvs.glob("**/clic*.csv"))
# all_csvs = list(csvs.glob("**/kodak*.csv"))


CONFIG_NUM_TO_LMBDA = {
    "00": 0.0001,
    "01": 0.0002,
    "02": 0.0004,
    "03": 0.001,
    "04": 0.004,
    "05": 0.02,
}

all_dfs = []
for file in all_csvs:
    df = pd.read_csv(file).assign(
        lmbda=CONFIG_NUM_TO_LMBDA[file.parent.parent.stem.split("_")[-1]]
    )
    all_dfs.append(df)
full_df = pd.concat(all_dfs).drop(
    columns=["mse", "anchor", "option_selected", "Unnamed: 0"]
)

img_dir = DATA_DIR / "clic20-pro-valid/"
pixels = []
for img in img_dir.glob("*.png"):
    # load image with pil and see how many pixels it has
    img_pil = Image.open(img)
    pixels.append({"seq_name": img.stem, "pixels": img_pil.size[0] * img_pil.size[1]})

pixels_df = pd.DataFrame(pixels)
full_df = full_df.merge(
    pixels_df,
    how="left",
    left_on="seq_name",
    right_on="seq_name",
)

In [ ]:
pd.concat(all_dfs).option_selected.value_counts().sort_index()

In [ ]:
full_df.head()

In [ ]:
from pathlib import Path

full_df.to_csv(Path("~/Downloads/all_clic.tsv").expanduser(), index=False, sep="\t")